# Svenskt Kvinnobiografiskt lexikon part 1
version 1.3.5
checks for new records in SKBL

* Denna [Jupyter Notebook](https://github.com/salgo60/open-data-examples/blob/master/Svenskt%20Kvinnobiografiskt%20lexikon.ipynb) 
  * [del 2](https://github.com/salgo60/open-data-examples/blob/master/Svenskt%20Kvinnobiografiskt%20lexikon%20part%202.ipynb) med sökfrågor etc.
  * [del 3](https://github.com/salgo60/open-data-examples/blob/master/Svenskt%20Kvinnobiografiskt%20lexikon%20part%203.ipynb) jmf SBL och SKBL och även SKBL <-> Wikidata
* Video om detta [svenska](https://www.youtube.com/watch?v=_LP48Yf6vgI) / [engelska](https://www.youtube.com/watch?v=wolyLu0WVsA)  
* Wikidata - [An introduction to Wikidata](https://www.youtube.com/watch?v=m_9_23jXPoE)
  * SKBL WIkidata property [P4963](https://www.wikidata.org/wiki/Property_talk:P4963) 
  

# Check diff SKBL Wikidata 
ett försök att hålla Wikidata / SKBL i fas 
* [KARP APIet](https://spraakbanken.gu.se/verktyg/karp/webb-api)
* se tidigare försök att ha ISNI i synk [T219700](https://phabricator.wikimedia.org/T219700), GITHUB [salgo60/SKBLWikidata](https://github.com/salgo60/SKBLWikidata) 
  * mar 2019 kommentar om [platser i SKBL](https://phabricator.wikimedia.org/T218782)  
  
Viss osäkerhet hur en bra API fråga skall se ut returnerar alla personer i SKBL

In [1]:
from datetime import datetime
print("Last run: ", datetime.now())


In [2]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 
pd.set_option("display.max.columns", None)
pd.set_option('display.max_rows', None)

  
#Ny bättre URL    
url = "https://ws.spraakbanken.gu.se/ws/karp/v5/query?mode=skbl&q=extended%7C%7Cand%7Cskblstatus%7Cequals%7Cok&size=10000"
r = http.request('GET', url) 
data = json.loads(r.data.decode('utf-8')) 
#print(type(data))
#print(r.info())  

listSKBL = []
i = 0 
for row in (data["hits"]["hits"]):
    #print (row["_source"]["url"])
    new_item = dict()
    new_item['skblid'] = row["_source"]["url"]
   
    try:
        publishedDate = row["_source"]['publication_date']        
    except:
        publishedDate =""
    new_item['publishedDate'] = publishedDate
    try:
        article_author_firstname = row["_source"]["article_author"][0]['firstname']
    except:
        article_author_firstname =""
    new_item['article_author_firstname'] = article_author_firstname
    try:
        article_author_lastname = row["_source"]["article_author"][0]['lastname']
    except:
        article_author_lastname =""
    new_item['article_author_lastname'] = article_author_lastname
    try:
        subtitle_sv = row["_source"]['subtitle']
    except:
        subtitle_sv =""
    new_item['subtitle_sv'] = subtitle_sv
    try:
        name_lastname = row["_source"]['name']['lastname']
    except:
        name_lastname = ""
    new_item['name_lastname'] = name_lastname
    try:
        name_firstname = row["_source"]['name']['firstname']
    except:
        name_firstname = ""
    new_item['name_firstname'] = name_firstname
    listSKBL.append(new_item)
    i = i + 1 
    #print (i)
print (len(listSKBL) ," antal poster")


1840  antal poster


In [3]:
#listSKBL

In [4]:
SKBLtot = pd.DataFrame(listSKBL,
                  columns=['skblid','publishedDate','article_author_firstname','article_author_lastname',
                          'subtitle_sv','name_lastname','name_firstname'])

In [5]:
SKBLtot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1840 entries, 0 to 1839
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   skblid                    1840 non-null   object
 1   publishedDate             1840 non-null   object
 2   article_author_firstname  1840 non-null   object
 3   article_author_lastname   1840 non-null   object
 4   subtitle_sv               1840 non-null   object
 5   name_lastname             1840 non-null   object
 6   name_firstname            1840 non-null   object
dtypes: object(7)
memory usage: 100.8+ KB


In [6]:
#SKBLtot

In [7]:
#SKBLtot.sort_index(inplace=True) 
SKBLtot.sort_values(['skblid'], ascending=[1],inplace=True) 

# Wikidata

In [8]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT * WHERE {
?item wdt:P31 wd:Q5.
?item wdt:P4963 ?skbl
} order by ?skbl"""


def get_results(endpoint_url, query):
    user_agent = "User:salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)
 
lst = []
for result in results["results"]["bindings"]:
    lst.append((result["skbl"]["value"]))
WDtot = pd.DataFrame(lst,columns={'skblid'}) 
WDtot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1824 entries, 0 to 1823
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   skblid  1824 non-null   object
dtypes: object(1)
memory usage: 14.4+ KB


In [9]:
WDtot.sort_index(inplace=True)  

In [10]:
WDtot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1824 entries, 0 to 1823
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   skblid  1824 non-null   object
dtypes: object(1)
memory usage: 14.4+ KB


In [11]:
#WDtot

### Check duplicates SKBL and Wikidata

In [12]:
dfmerge = pd.merge(WDtot, SKBLtot,how='outer', indicator=True)

In [13]:
# check whatsnew / deleted  
dfmerge['_merge'].value_counts()

both          1824
right_only      16
left_only        0
Name: _merge, dtype: int64

In [14]:
#dfmerge

In [15]:
SKBLnew = dfmerge[dfmerge['_merge']=="right_only"]


In [16]:
SKBLnew

,skblid,publishedDate,article_author_firstname,article_author_lastname,subtitle_sv,name_lastname,name_firstname,_merge
1824,AmyReginaNitaHarleman,2021-01-31,Fredrik,Tersmeden,"Skådespelare, revyartist, sufflör",Hårleman,Amy Regina (/Nita/),right_only
1825,BirgitSunesson,2021-01-31,Birgitta,Ney,"Journalist, författare",Sunesson,/Birgit/ Sofia,right_only
1826,ElsaSofiaHammarMoeschlin,2021-02-02,Christian,Mehrstam,"Konstnär, författare",Hammar-Moeschlin,/Elsa/ Sofia,right_only
1827,ElsaSvartengren,2021-01-31,Birgitta,Wistrand,"Politisk journalist, feminist",Svartengren,/Elsa/ Maria,right_only
1828,EmmaAmaliaZorn,2021-02-02,Birgitta,Sandström,"Filantrop, donator, museigrundare",Zorn,/Emma/ Amalia,right_only
1829,ErnaMarianneEriksson,2021-01-31,Caroline,Bergström,"Redaktör, förlagschef",Eriksson,Erna /Marianne/,right_only
1830,GerdaMargaretaRenberg,2021-01-31,Birgitta,Flensburg,"Konstnär, poet",Renberg,Gerda /Margareta/,right_only
1831,HedvigAmaliaPosse,2021-02-02,Kristina,Helgesson Kjellin,"Missionär, sjukhusgrundare, psalmförfattare",Posse,/Hedvig/ Amalia,right_only
1832,HildegardDinclau,2021-02-01,Johanna,Rosenqvist,"Textilkonstnär, lärare",Dinclau,/Hildegard/ Maria,right_only
1833,IngaStinaGill,2021-02-01,Jennifer,Vintkvist,"Skådespelare, dansare",Gill,/Inga/ Stina,right_only


In [17]:
SKBLnew.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 1824 to 1839
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   skblid                    16 non-null     object  
 1   publishedDate             16 non-null     object  
 2   article_author_firstname  16 non-null     object  
 3   article_author_lastname   16 non-null     object  
 4   subtitle_sv               16 non-null     object  
 5   name_lastname             16 non-null     object  
 6   name_firstname            16 non-null     object  
 7   _merge                    16 non-null     category
dtypes: category(1), object(7)
memory usage: 1.1+ KB


In [18]:
import csv  
jsonURL = "https://skbl.se/sv/artikel/"
listNewItems =[]
for index,row in SKBLnew.iterrows():
    url = jsonURL + row["skblid"] + ".json"
    r = http.request('GET', url) 
    data = json.loads(r.data.decode('utf-8'))  
    
    new_item = dict()
    try:
        publishedDate = data['publication_date'],        
    except:
        publishedDate =""
    new_item['publishedDate'] = publishedDate[0]
    try:
        birthdate = data['lifespan']['from']['date']['date'],
    except:
        birtdate =""
    new_item['birthdate'] = birthdate[0]
    try:
        deathdate = data['lifespan']['to']['date']['date']
    except:
        deathdate=""
    new_item['deathdate'] = deathdate
    try:
        birthparish = data['lifespan']['from']['place']['parish']
    except:
        birthparish = ""        
    new_item['birthparish'] = birthparish
    try:
        deathparish = data['lifespan']['to']['place']['parish']
    except:
        deathparish = ""
    new_item['deathparish'] = deathparish
    try:
        ISNI = data['id']['id']
    except:
        ISNI = ""
    new_item['ISNI'] = ISNI
    new_item['SKBL'] = row["skblid"]
    new_item['index'] = index
    new_item['skbl_status'] = data['skbl_status']
    new_item['firstname'] = data['name'].get('firstname')
    new_item['lastname'] = data['name'].get('lastname')
    new_item['subtitle'] = data['subtitle']
    listNewItems.append(new_item)
print (len(listNewItems) ," antal poster")
if len(listNewItems) > 0:
    keys = listNewItems[0].keys()
    with open("SKBL_newfile.csv", "w", newline='') as SKBLfile:
        dict_writer = csv.DictWriter(SKBLfile, keys)
        dict_writer.writeheader()
        dict_writer.writerows(listNewItems)

    dfSKBLcsv = pd.read_csv("SKBL_newfile.csv", sep=",")   
    dfSKBLcsv
else:
    print ("Ingen fil skapas inga nya poster") 
            


16  antal poster


In [19]:
end = datetime.now()
print("Ended: ", end)

Ended:  2021-02-02 18:29:29.365415
